# Dynamic Programming
Basically we solve the Bellman optimality equation using these methods:
* Value Iteration
* Policy Iteration

In [21]:
from grid_world.grid_samples import *
from grid_world.grid_actions import GridActions
from grid_world.gridworld_mdp import *

In [22]:
grid_string = get_book_grid()
print(grid_string)

[[' ', ' ', ' ', 1], [' ', '#', ' ', -1], ['S', ' ', ' ', ' ']]


In [23]:
grid_world = GridWorld(grid_string)
print('Grid shape:', grid_world.shape)
print('All actions:', grid_world.all_actions)
print('Number of states:', grid_world.num_states)
print('States:', grid_world.states)
print('Start state:', grid_world.start_state)
print('Rewards on each state')
for st in grid_world.states:
    print('\tState:' , st,'Reward:', grid_world.R(st))

Grid shape: (3, 4)
All actions: ['up', 'down', 'left', 'right']
Number of states: 11
States: {(0, 1), (1, 2), (0, 0), (1, 3), (2, 1), (2, 0), (2, 3), (2, 2), (1, 0), (0, 2), (0, 3)}
Start state: (2, 0)
Rewards on each state
	State: (0, 1) Reward: 0
	State: (1, 2) Reward: 0
	State: (0, 0) Reward: 0
	State: (1, 3) Reward: -1
	State: (2, 1) Reward: 0
	State: (2, 0) Reward: 0
	State: (2, 3) Reward: 0
	State: (2, 2) Reward: 0
	State: (1, 0) Reward: 0
	State: (0, 2) Reward: 0
	State: (0, 3) Reward: 1


### Solve with Value iteration

In [24]:
value_mdp = value_iteration(grid_world)
policy = best_policy(grid_world, value_mdp)
print('Value:',value_mdp)
print('Policy:')
for st in grid_world.states:
    print('\tState:', st, 'action:', GridActions.action_to_str(policy[st]))

Value: {(0, 1): 0.7443801180533612, (1, 2): 0.5718590147306759, (0, 0): 0.644967826744644, (2, 1): 0.43075218166042545, (0, 2): 0.8477662714927858, (2, 0): 0.49065027469590516, (1, 3): -1.0, (2, 3): 0.27724220363051516, (2, 2): 0.4754426095304643, (1, 0): 0.5663098424341917, (0, 3): 1.0}
Policy:
	State: (0, 1) action: right
	State: (1, 2) action: up
	State: (0, 0) action: right
	State: (1, 3) action: None
	State: (2, 1) action: left
	State: (2, 0) action: up
	State: (2, 3) action: left
	State: (2, 2) action: up
	State: (1, 0) action: up
	State: (0, 2) action: right
	State: (0, 3) action: None
